In [1]:
#トラカンデータの距離案分()
import numpy as np
import psycopg2
import datetime
import pandas as pd

In [136]:
#計測地点のosm_idを指定
start_osm = 315917801
mid_osm = []
end_osm = 315499982
start_mesh = 50325623342
end_mesh = 50325613134


In [137]:
connection = psycopg2.connectconnection = psycopg2.connect(host="localhost", database="analysisdb", user="postgres",password="setouchi1311")

In [138]:
getdbname = "kddi_12_new_param"
dbname = "kddi_trkn12_monthly_anbun_3b_26"
trknName = "trkn_12_grp"
sttime = "2022-04-01 12:00:00"

In [139]:
datedir = "230225_1"
f = open(datedir+'/'+ dbname + '.csv', 'w')
for i in range(30):
    dte = datetime.datetime.strptime(sttime, '%Y-%m-%d %H:%M:%S')
    if i != 0:
        dte = dte + datetime.timedelta(days=i)
    for j in range(24):
        timestr = dte.strftime('%H:%M:%S')
        #特定日付のKDDIメッシュ情報を全取得
        cur =connection.cursor()
        sql = "SELECT mesh, tbran,pointdata.latitude,pointdata.longitude FROM " + getdbname + " "
        sql += "INNER JOIN pointdata ON mesh = cast(pointdata.grid_code AS BIGINT)  "
        sql += "WHERE times = '"+str(dte)+"' AND pointdata.grid_code != '0'  ORDER BY mesh "
        cur.execute(sql)
        kddiDf = pd.DataFrame.from_records(cur.fetchall(), columns=["mesh","tbran","latitude","longitude"])
        cur.close()

        one_target_list = []
        one_target_list.append(start_mesh)
        one_target_list.append(end_mesh)
        two_target_list = []
        two_target_list.append(start_osm)
        if len(mid_osm) != 0:
            for target in mid_osm:
                two_target_list.append(target)
        if start_osm != end_osm:
            two_target_list.append(end_osm)

        cur =connection.cursor()
        sql = "SELECT latitude,longitude FROM pointdata WHERE pointdata.grid_code = '"+str(start_mesh)+"'"
        cur.execute(sql)
        for row in cur:
            stlat = row[0]
            stlng = row[1]
        sql = "SELECT latitude,longitude FROM pointdata WHERE pointdata.grid_code = '"+str(end_mesh)+"'"
        cur.execute(sql)
        for row in cur:
            edlat = row[0]
            edlng = row[1]
        cur.close()

        if stlat > edlat:
            wklat = stlat
            stlat = edlat
            edlat = wklat
        if stlng > edlng:
            wklng = stlng
            stlng = edlng
            edlng = wklng

        print(str(stlat) + ":" + str(edlat) + ":" + str(stlng) + ":" + str(stlng))

        keylist = []
        cnt = 0
        for item in two_target_list:
            sql = "SELECT a.grid_code,a.latitude,a.longitude"
            sql += " FROM pointdata a, roadmap r"
            sql += " WHERE osm_id = "+ str(item) +" and ST_Distance(a.geom, r.geom) < 0.001"
            sql += " order by ST_Distance(a.geom, r.geom);"
            cur =connection.cursor()
            cur.execute(sql)
            childlist = []
            for row in cur:
                if row[1] >= stlat and row[1] <= edlat and row[2] >= stlng and row[2] <= edlng:
                    childlist.append([row[0], row[1], row[2]])
                #print(row[0])
            worklist = [item, childlist]
            keylist.append(worklist)
            cnt = cnt + 1
            cur.close()
        #特定日付のトラカンデータを全取得 > 緯度経度
        cur =connection.cursor()
        #sql = "SELECT trakanmap.gridcode , 断面交通量,pointdata.latitude,pointdata.longitude FROM trakan2204 "
        #sql += "INNER JOIN trakanmap ON trakan2204.計測地点番号 = trakanmap.地点番号 "
        #sql += "INNER JOIN pointdata ON trakanmap.gridcode = pointdata.grid_code "
        #sql += "WHERE 時刻 = '"+str(dte)+"' AND (trakanmap.gridcode = '"+str(start_mesh)+"' or trakanmap.gridcode = '"+str(end_mesh)+"') "
        #sql += "ORDER BY trakanmap.gridcode"
        sql = "SELECT grid_code, tbran, latitude, longitude FROM " + trknName  + " "
        sql += "INNER JOIN pointdata ON mesh = cast(grid_code as bigint) "
        sql += "WHERE times = '"+timestr+"' AND (mesh = '"+str(start_mesh)+"' or mesh = '"+str(end_mesh)+"') "
        sql += "ORDER BY mesh"
        #print(sql)
        cur.execute(sql)
        trknDf = pd.DataFrame(cur.fetchall(), columns=["gridcode","danmen","latitude","longitude"])
        cur.close()
        trknDf
        #keylist

        from geopy.distance import geodesic

        #データが存在する道路IDごとに交通量を平均化して取得する
        lstItems = []
        cntDt = 0
        wks = ""
        wknp=""
        find = 0

        trknStParam = trknDf.query('gridcode == "'+str(start_mesh) + '"')
        trknEdParam = trknDf.query('gridcode == "'+str(end_mesh) + '"')
        trknStParam

        if len(trknStParam) == 0:
            print(str(start_mesh) + " mesh empty!")
            dte = dte + datetime.timedelta(minutes=15) 
            continue
        elif len(trknStParam) == 1:
            trknSt = trknStParam['danmen'].iloc[-1]
        else:
            trknSt = trknStParam['danmen'].iloc[0] + trknStParam['danmen'].iloc[1]

        if len(trknEdParam) == 0:
            print(str(end_mesh) + " mesh empty!")
            dte = dte + datetime.timedelta(minutes=15) 
            continue
        elif len(trknEdParam) == 1:
            trknEd = trknEdParam['danmen'].iloc[-1]
        else:
            trknEd = trknEdParam['danmen'].iloc[0] + trknEdParam['danmen'].iloc[1]

        st_station = [trknStParam['latitude'].iloc[-1], trknStParam['longitude'].iloc[-1]]
        ed_station = [trknEdParam['latitude'].iloc[-1], trknEdParam['longitude'].iloc[-1]]

        dis_km = geodesic(st_station, ed_station)
        #cur =connection.cursor()
        for row in keylist:
            for meshid in row[1]:
                #路線のメッシュ数分ループ
                #print(str(meshid[0]))
                kddiParam = kddiDf.query('mesh == '+str(meshid[0]))
                if len(kddiParam) == 0:
                    print(str(meshid[0]) + " is kddi none!")
                    continue
                #路線のメッシュがkddiに存在したなら
                
                kddi_station = [kddiParam['latitude'].iloc[-1], kddiParam['longitude'].iloc[-1]]
                kddi_km = geodesic(st_station, kddi_station)
                
                if trknSt < trknEd:
                    ret = ((trknEd - trknSt) * (kddi_km / dis_km)) + trknSt
                else:
                    ret = trknSt - ((trknEd - trknSt) * (kddi_km / dis_km))
                print("存在メッシュ：" + str(meshid[0]) + "値" + str(ret) + "距離" + str(kddi_km))
                #sql = "INSERT INTO "+dbname+" (times, stmesh, edmesh, mesh, tbran) VALUES ('"+str(dte)+"',"+str(start_mesh)+","  +str(end_mesh)+","  + str(meshid[0]) + ","  +  str(ret) + ") ON CONFLICT DO NOTHING;"
                f.write(str(dte) +  "," + str(start_mesh)+","  +str(end_mesh)+","  + str(meshid[0]) + ","  +  str(ret) + '\n')
                #cur.execute(sql)
                #connection.commit()

        #sql = "INSERT INTO "+dbname+" (times, stmesh, edmesh, mesh, tbran) VALUES ('"+str(dte)+"',"+str(start_mesh)+","  +str(end_mesh)+","  + str(start_mesh) + ","  + str(trknSt) + ") ON CONFLICT DO NOTHING;"
        f.write(str(dte) +  "," + str(start_mesh)+","  +str(end_mesh)+","  + str(meshid[0]) + ","  +  str(ret) + '\n')
        #cur.execute(sql)
        #connection.commit()
        #cur.close()
        dte = dte + datetime.timedelta(minutes=15) 
        #print(lstItems)
f.close()

33.7619791665:33.7734375000:132.789843750:132.789843750
存在メッシュ：50325613334値27.589258499532068距離0.8590233749368513 km
存在メッシュ：50325623342値16.06779661016949距離0.0 km
存在メッシュ：50325623321値19.724853517378442距離0.2726648230002139 km
存在メッシュ：50325623114値24.734376072599265距離0.6461675098581355 km
存在メッシュ：50325623132値23.381933845148314距離0.5453313922032513 km
存在メッシュ：50325623112値26.14377915162449距離0.7512505454326793 km
存在メッシュ：50325623324値17.61744965245375距離0.11553986804341464 km
存在メッシュ：50325623134値22.12473140428584距離0.4515962139681325 km
存在メッシュ：50325623143値21.105775391742597距離0.3756243415432246 km
存在メッシュ：50325623323値18.55169334278078距離0.1851957134200454 km
存在メッシュ：50325613332値29.058777915051177距離0.9685885967790505 km
存在メッシュ：50325613312値32.04331144442091距離1.1911110587401013 km
存在メッシュ：50325613314値30.545017363738623距離1.07940043987984 km
存在メッシュ：50325613134値33.550561797752806距離1.303489444211249 km
存在メッシュ：50325613334値27.589258499532068距離0.8590233749368513 km
存在メッシュ：50325613332値29.058777915051177距離0.96858859677

33.7619791665:33.7734375000:132.789843750:132.789843750
存在メッシュ：50325613334値25.082275806735495距離0.8590233749368513 km
存在メッシュ：50325623342値15.1距離0.0 km
存在メッシュ：50325623321値18.268499886493704距離0.2726648230002139 km
存在メッシュ：50325623114値22.608785545247308距離0.6461675098581355 km
存在メッシュ：50325623132値21.4370200647265距離0.5453313922032513 km
存在メッシュ：50325623112値23.82990230914352距離0.7512505454326793 km
存在メッシュ：50325623324値16.44263032081982距離0.11553986804341464 km
存在メッシュ：50325623134値20.347771006742192距離0.4515962139681325 km
存在メッシュ：50325623143値19.464940333880346距離0.3756243415432246 km
存在メッシュ：50325623323値17.252065640495452距離0.1851957134200454 km
存在メッシュ：50325613332値26.3554777883875距離0.9685885967790505 km
存在メッシュ：50325613312値28.941298679061525距離1.1911110587401013 km
存在メッシュ：50325613314値27.643166124652033距離1.07940043987984 km
存在メッシュ：50325613134値30.247191011235955距離1.303489444211249 km
存在メッシュ：50325613334値25.082275806735495距離0.8590233749368513 km
存在メッシュ：50325613332値26.3554777883875距離0.9685885967790505 km
33.7619

33.7619791665:33.7734375000:132.789843750:132.789843750
存在メッシュ：50325613334値26.669209431689453距離0.8590233749368513 km
存在メッシュ：50325623342値16.516666666666666距離0.0 km
存在メッシュ：50325623321値19.739211427140198距離0.2726648230002139 km
存在メッシュ：50325623114値24.153529025255295距離0.6461675098581355 km
存在メッシュ：50325623132値22.961776825467552距離0.5453313922032513 km
存在メッシュ：50325623112値25.395474289698644距離0.7512505454326793 km
存在メッシュ：50325623324値17.88219813600775距離0.11553986804341464 km
存在メッシュ：50325623134値21.85394852488256距離0.4515962139681325 km
存在メッシュ：50325623143値20.956059472923485距離0.3756243415432246 km
存在メッシュ：50325623323値18.70543993553625距離0.1851957134200454 km
存在メッシュ：50325613332値27.964128327690286距離0.9685885967790505 km
存在メッシュ：50325613312値30.59405537874847距離1.1911110587401013 km
存在メッシュ：50325613314値29.27378067100919距離1.07940043987984 km
存在メッシュ：50325613134値31.92222222222222距離1.303489444211249 km
存在メッシュ：50325613334値26.669209431689453距離0.8590233749368513 km
存在メッシュ：50325613332値27.964128327690286距離0.96858859677

33.7619791665:33.7734375000:132.789843750:132.789843750
存在メッシュ：50325613334値26.076044488567156距離0.8590233749368513 km
存在メッシュ：50325623342値14.583333333333334距離0.0 km
存在メッシュ：50325623321値18.231264395936577距離0.2726648230002139 km
存在メッシュ：50325623114値23.228286154602223距離0.6461675098581355 km
存在メッシュ：50325623132値21.879218820656916距離0.5453313922032513 km
存在メッシュ：50325623112値24.634172199626516距離0.7512505454326793 km
存在メッシュ：50325623324値16.129119360907275距離0.11553986804341464 km
存在メッシュ：50325623134値20.625153611290784距離0.4515962139681325 km
存在メッシュ：50325623143値19.60874030849105距離0.3756243415432246 km
存在メッシュ：50325623323値17.061031733194113距離0.1851957134200454 km
存在メッシュ：50325613332値27.541896855371583距離0.9685885967790505 km
存在メッシュ：50325613312値30.518982759533223距離1.1911110587401013 km
存在メッシュ：50325613314値29.02442753206019距離1.07940043987984 km
存在メッシュ：50325613134値32.02247191011236距離1.303489444211249 km
存在メッシュ：50325613334値26.076044488567156距離0.8590233749368513 km
存在メッシュ：50325613332値27.541896855371583距離0.96858859

33.7619791665:33.7734375000:132.789843750:132.789843750
存在メッシュ：50325613334値24.59642405778512距離0.8590233749368513 km
存在メッシュ：50325623342値13.916666666666666距離0.0 km
存在メッシュ：50325623321値17.30655598015433距離0.2726648230002139 km
存在メッシュ：50325623114値21.950106082462923距離0.6461675098581355 km
存在メッシュ：50325623132値20.696467003205502距離0.5453313922032513 km
存在メッシュ：50325623112値23.256544718013323距離0.7512505454326793 km
存在メッシュ：50325623324値15.353109254502204距離0.11553986804341464 km
存在メッシュ：50325623134値19.531109934036216距離0.4515962139681325 km
存在メッシュ：50325623143値18.58659411558522距離0.3756243415432246 km
存在メッシュ：50325623323値16.219101439581408距離0.1851957134200454 km
存在メッシュ：50325613332値25.95858720947442距離0.9685885967790505 km
存在メッシュ：50325613312値28.725084579808616距離1.1911110587401013 km
存在メッシュ：50325613314値27.33624890635911距離1.07940043987984 km
存在メッシュ：50325613134値30.12222222222222距離1.303489444211249 km
存在メッシュ：50325613334値24.59642405778512距離0.8590233749368513 km
存在メッシュ：50325613332値25.95858720947442距離0.9685885967790

...

33.7619791665:33.7734375000:132.789843750:132.789843750
存在メッシュ：50325613334値26.076044488567156距離0.8590233749368513 km
存在メッシュ：50325623342値14.583333333333334距離0.0 km
存在メッシュ：50325623321値18.231264395936577距離0.2726648230002139 km
存在メッシュ：50325623114値23.228286154602223距離0.6461675098581355 km
存在メッシュ：50325623132値21.879218820656916距離0.5453313922032513 km
存在メッシュ：50325623112値24.634172199626516距離0.7512505454326793 km
存在メッシュ：50325623324値16.129119360907275距離0.11553986804341464 km
存在メッシュ：50325623134値20.625153611290784距離0.4515962139681325 km
存在メッシュ：50325623143値19.60874030849105距離0.3756243415432246 km
存在メッシュ：50325623323値17.061031733194113距離0.1851957134200454 km
存在メッシュ：50325613332値27.541896855371583距離0.9685885967790505 km
存在メッシュ：50325613312値30.518982759533223距離1.1911110587401013 km
存在メッシュ：50325613314値29.02442753206019距離1.07940043987984 km
存在メッシュ：50325613134値32.02247191011236距離1.303489444211249 km
存在メッシュ：50325613334値26.076044488567156距離0.8590233749368513 km
存在メッシュ：50325613332値27.541896855371583距離0.96858859

33.7619791665:33.7734375000:132.789843750:132.789843750
存在メッシュ：50325613334値24.59642405778512距離0.8590233749368513 km
存在メッシュ：50325623342値13.916666666666666距離0.0 km
存在メッシュ：50325623321値17.30655598015433距離0.2726648230002139 km
存在メッシュ：50325623114値21.950106082462923距離0.6461675098581355 km
存在メッシュ：50325623132値20.696467003205502距離0.5453313922032513 km
存在メッシュ：50325623112値23.256544718013323距離0.7512505454326793 km
存在メッシュ：50325623324値15.353109254502204距離0.11553986804341464 km
存在メッシュ：50325623134値19.531109934036216距離0.4515962139681325 km
存在メッシュ：50325623143値18.58659411558522距離0.3756243415432246 km
存在メッシュ：50325623323値16.219101439581408距離0.1851957134200454 km
存在メッシュ：50325613332値25.95858720947442距離0.9685885967790505 km
存在メッシュ：50325613312値28.725084579808616距離1.1911110587401013 km
存在メッシュ：50325613314値27.33624890635911距離1.07940043987984 km
存在メッシュ：50325613134値30.12222222222222距離1.303489444211249 km
存在メッシュ：50325613334値24.59642405778512距離0.8590233749368513 km
存在メッシュ：50325613332値25.95858720947442距離0.9685885967790

33.7619791665:33.7734375000:132.789843750:132.789843750
存在メッシュ：50325613334値26.32652707226226距離0.8590233749368513 km
存在メッシュ：50325623342値16.220338983050848距離0.0 km
存在メッシュ：50325623321値19.42817018642674距離0.2726648230002139 km
存在メッシュ：50325623114値23.82233280800787距離0.6461675098581355 km
存在メッシュ：50325623132値22.63602193342774距離0.5453313922032513 km
存在メッシュ：50325623112値25.058607574779735距離0.7512505454326793 km
存在メッシュ：50325623324値17.579635682439562距離0.11553986804341464 km
存在メッシュ：50325623134値21.533251776690733距離0.4515962139681325 km
存在メッシュ：50325623143値20.639462323904148距離0.3756243415432246 km
存在メッシュ：50325623323値18.399118708662225距離0.1851957134200454 km
存在メッシュ：50325613332値27.61553360256167距離0.9685885967790505 km
存在メッシュ：50325613312値30.23345288190385距離1.1911110587401013 km
存在メッシュ：50325613314値28.91920630992087距離1.07940043987984 km
存在メッシュ：50325613134値31.555555555555557距離1.303489444211249 km
存在メッシュ：50325613334値26.32652707226226距離0.8590233749368513 km
存在メッシュ：50325613332値27.61553360256167距離0.96858859677905

33.7619791665:33.7734375000:132.789843750:132.789843750
存在メッシュ：50325613334値27.144424582107284距離0.8590233749368513 km
存在メッシュ：50325623342値15.11864406779661距離0.0 km
存在メッシュ：50325623321値18.93577804929623距離0.2726648230002139 km
存在メッシュ：50325623114値24.164577945444417距離0.6461675098581355 km
存在メッシュ：50325623132値22.752936476523985距離0.5453313922032513 km
存在メッシュ：50325623112値25.635673562242808距離0.7512505454326793 km
存在メッシュ：50325623324値16.73612868379401距離0.11553986804341464 km
存在メッシュ：50325623134値21.44070364000985距離0.4515962139681325 km
存在メッシュ：50325623143値20.377145779188755距離0.3756243415432246 km
存在メッシュ：50325623323値17.71126618584359距離0.1851957134200454 km
存在メッシュ：50325613332値28.678267953916468距離0.9685885967790505 km
存在メッシュ：50325613312値31.79344079452091距離1.1911110587401013 km
存在メッシュ：50325613314値30.22956322992021距離1.07940043987984 km
存在メッシュ：50325613134値33.36666666666667距離1.303489444211249 km
存在メッシュ：50325613334値27.144424582107284距離0.8590233749368513 km
存在メッシュ：50325613332値28.678267953916468距離0.9685885967790

33.7619791665:33.7734375000:132.789843750:132.789843750
存在メッシュ：50325613334値24.396201416819686距離0.8590233749368513 km
存在メッシュ：50325623342値15.133333333333333距離0.0 km
存在メッシュ：50325623321値18.073484160595516距離0.2726648230002139 km
存在メッシュ：50325623114値22.100971907884084距離0.6461675098581355 km
存在メッシュ：50325623132値21.013653817201437距離0.5453313922032513 km
存在メッシュ：50325623112値23.23408460858429距離0.7512505454326793 km
存在メッシュ：50325623324値16.37920229021503距離0.11553986804341464 km
存在メッシュ：50325623134値20.002905313602433距離0.4515962139681325 km
存在メッシュ：50325623143値19.183698930098483距離0.3756243415432246 km
存在メッシュ：50325623323値17.130302814126715距離0.1851957134200454 km
存在メッシュ：50325613332値25.57764563134623距離0.9685885967790505 km
存在メッシュ：50325613312値27.97711026862615距離1.1911110587401013 km
存在メッシュ：50325613314値26.772532190522874距離1.07940043987984 km
存在メッシュ：50325613134値29.18888888888889距離1.303489444211249 km
存在メッシュ：50325613334値24.396201416819686距離0.8590233749368513 km
存在メッシュ：50325613332値25.57764563134623距離0.96858859677

In [140]:
connection.close()

In [130]:
#CreateGeojsonFile